In [2]:
import pandas as pd
import numpy as np
import os
import re

from keras.utils import to_categorical
from keras.optimizers import Adam

import tensorflow as tf

import transformers
from transformers import DistilBertTokenizer, TFDistilBertModel

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

MAX_SEQUENCE_LENGTH = 66
BATCH_SIZE = 32
CATEGORIES = "sadness,joy,anger,fear,surprise".split(',')


def tokenize(sentences, max_length=MAX_SEQUENCE_LENGTH, padding='max_length'):

    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf"
    )

def decode(tokens):
    return tokenizer.decode(tokens)
def to_tensor(data, label = []):
    return tf.data.Dataset.from_tensor_slices((
    dict(tokenize(data)),  # Convert BatchEncoding instance to dictionary
    label
    )).batch(BATCH_SIZE).prefetch(1)
def predict(str,model):
    str = to_tensor(str)
    model.predict(str)
def predice(array,model):
    array = tf.data.Dataset.from_tensor_slices((
    dict(tokenize(array)),
    )).batch(BATCH_SIZE).prefetch(1)
    return pd.DataFrame(columns=CATEGORIES,data=model.predict(array))

In [5]:
new_model = tf.keras.models.load_model('SentimentAnalysis.hdf5', custom_objects={'TFDistilBertModel': TFDistilBertModel})

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [6]:
predice('I love you',new_model)

1/1 [==============================] - 3s 3s/step


,sadness,joy,anger,fear,surprise
0,0.003098,0.993538,0.002981,0.000144,0.000239


In [50]:
model = whisper.load_model('medium')


In [54]:
!whisper "prueba.mp3" --model medium --language es --task transcribe --device cpu

C:\Users\mario\anaconda3\envs\entornoGPU\lib\site-packages\whisper\transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Traceback (most recent call last):
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\mario\anaconda3\envs\entornoGPU\Scripts\whisper.exe\__main__.py", line 7, in <module>
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\site-packages\whisper\transcribe.py", line 307, in cli
    result = transcribe(model, audio_path, temperature=temperature, **args)
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\site-packages\whisper\transcribe.py", line 84, in transcribe
    mel = log_mel_spectrogram(audio)
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\s

In [55]:
#@title ⚙️ Ejecutar esta celda para instalar las librería.
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer

from IPython.display import HTML, Audio
from js2py import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

  Cloning https://github.com/openai/whisper.git to c:\users\mario\appdata\local\temp\pip-req-build-hqz17knx
  Resolved https://github.com/openai/whisper.git to commit eff383b27b783e280c089475852ba83f20f64998
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\mario\AppData\Local\Temp\pip-req-build-hqz17knx'


In [63]:
import numpy as np
import whisper
from scipy.io.wavfile import write
from IPython.display import clear_output



!whisper "C:/Users/mario/prueba.mp3" --task transcribe --model medium --language es




C:\Users\mario\anaconda3\envs\entornoGPU\lib\site-packages\whisper\transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Traceback (most recent call last):
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\mario\anaconda3\envs\entornoGPU\Scripts\whisper.exe\__main__.py", line 7, in <module>
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\site-packages\whisper\transcribe.py", line 307, in cli
    result = transcribe(model, audio_path, temperature=temperature, **args)
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\site-packages\whisper\transcribe.py", line 84, in transcribe
    mel = log_mel_spectrogram(audio)
  File "C:\Users\mario\anaconda3\envs\entornoGPU\lib\s

In [65]:
!conda install -c conda-forge ffmpeg


^C
